# AB Teste Course - PA James

## 0.1. Imports

In [1]:
import pandas as pd
import math
import numpy as np
from statsmodels.stats import api as sms


## 1.0. Load Data

In [2]:

df_raw = pd.read_csv("datasets/ab_data.csv")

In [3]:
df_raw

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


## 2.0. Design de Experimentos

### 2.1. Formulação das hipoteses

### 2.2. Parâmetros do Experimento

In [4]:
# nivel de confiança
confidence_level = 0.95

# nivel de significancia
significanse_level = 1 - confidence_level

# conversóes da página atual e da nova pagina
p1 = 0.13
p2 = 0.15

# tamanho do efeito
effect_size = sms.proportion_effectsize(p1, p2)

# Poder estatístico
power = 0.80

# sample size
sample_n = sms.NormalIndPower().solve_power(
                effect_size,
                power=power,
                alpha=significanse_level

)

sample_n = math.ceil(sample_n)
sample_n

4720

In [5]:
conversion_rate = 0.10
x = sample_n / conversion_rate
print("O numero de email para enviar: {}".format(int(x)))
print("O numero total da base de email para enviar: {}".format(int(2*x)))

O numero de email para enviar: 47200
O numero total da base de email para enviar: 94400


In [6]:
print("O tamanho total da amostra é de {}".format( 2*sample_n))
print("O tamanho da amostra do grupo controle é de: {}".format( sample_n))
print("O tamanho da amostra do grupo tratamento é de: {}".format( sample_n))

O tamanho total da amostra é de 9440
O tamanho da amostra do grupo controle é de: 4720
O tamanho da amostra do grupo tratamento é de: 4720


In [7]:
## Anásise descritiva dos dados
print(f"Number of rows: {df_raw.shape[0]}")
print(f"Number of cols: {df_raw.shape[1]}")


Number of rows: 294478
Number of cols: 5


In [8]:
# Verificar dados faltantes
df_raw.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [9]:
#3.2 Conferir as flags do teste A
df_user_delete = df_raw[['user_id', 'group']].groupby('user_id').count().reset_index().query('group > 1')['user_id']
df1 = df_raw[~df_raw['user_id'].isin(df_user_delete)]
df1.shape

(286690, 5)

In [10]:
#data without duplicated
df1[['user_id','group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,old_page,143293
1,treatment,new_page,143397


In [13]:
# 3.3 Amostragem aleatória dos grupos Controle e Tratamento
# Control group
df_control_sample = df1[df1['group'] == 'control'].sample(n=sample_n, random_state=42)
print(f'Size of Control group: {df_control_sample.shape[0]}')

# Treatment group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample(n=sample_n, random_state=42)
print(f'Size of tratment group: {df_treatment_sample.shape[0]}')

# Total Sample Size
df_ab = pd.concat([df_control_sample, df_treatment_sample]).reset_index(drop=True)
print(f'Total sample size: {df_ab.shape[0]}')


Size of Control group: 4720
Size of tratment group: 4720
Total sample size: 9440


In [14]:
#Cálculo da métrica de interesse entre os groupos (Conversão de cada página)

#Control Group
sales = df_control_sample.loc[df_control_sample['converted'] == 1, 'converted'].sum()
visit = len(df_control_sample)

conversion_rate_control = sales / visit
print(f'Converstion Rate - Control Goup: {conversion_rate_control:.4f}')

#Treatment Group
sales = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
visit = len(df_treatment_sample)

conversion_rate_treatment = sales / visit
print(f'Converstion Rate - treatment Goup: {conversion_rate_treatment:.4f}')


# conversão = compradores / visitantes


Converstion Rate - Control Goup: 0.1155
Converstion Rate - treatment Goup: 0.1290
